In [ ]:
import holoviews as hv

from hv_anndata.interface import ACCESSOR as A, register

register()

hv.extension('bokeh')

## Create synthetic data

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from anndata import AnnData
from string import ascii_lowercase

gen = np.random.default_rng()
x = gen.random((100, 50), dtype=np.float32)
layers = dict(a=sp.random(100, 50, format="csr"))
obs = pd.DataFrame(
    dict(type=gen.integers(0, 3, size=100)),
    index=pd.Series(range(100)).astype(str).apply(lambda v: 'cell-'+v),
)
var_grp = pd.Categorical(
    gen.integers(0, 6, size=50), categories=list(ascii_lowercase[:5])
)
var = pd.DataFrame(
    dict(grp=var_grp),
    index=pd.Series(range(50)).astype(str).apply(lambda v: 'gene-'+v),
)
obsm = dict(umap=gen.random((100, 2)))
varp = dict(cons=sp.csr_array(sp.random(50, 50)))
adata = AnnData(x, obs, var, layers=layers, obsm=obsm, varm={}, obsp={}, varp=varp)

## Tabular mode

We can select the data to display using the `Accessor` object `A`. Here it is important that the data must be indexed along either the `obs` or the `var` dimension.

Let us visualize the UMAP data to start:

In [ ]:
scatter = hv.Scatter(adata, A.obsm['umap'][0], [A.obsm['umap'][1], A.obs['type']]).opts(color="A.obs['type']")

scatter

To make it easier to access data we can also specify the dimensions as strings:

In [ ]:
hv.Scatter(adata, 'obsm.umap.0', ['obsm.umap.1', 'obs.type']).opts(color="A.obs['type']")

For an `obs` indexed dataset you can use the select method along the obs variables, e.g. we can select a specific type:

In [ ]:
scatter.select(**{'obs.type': 2})

We can also use the `.iloc` method to index along the obs dimension:

In [ ]:
scatter.iloc[:50]

We cannot can mix and match `obs` and `var` dimensions:

In [ ]:
hv.Scatter(adata, A.obsm['umap'][0], A.var['index'])

## Gridded Data

AnnData can also hold gridded data and we can render that.

When rendering an Element that assumes continuous coordinates (e.g. `Image`) it will render the integer indices:

In [ ]:
img = hv.Image(adata, [A.var['index'], A.obs['index']], [A[:, :]])

img

When rendering into an element that assumes discrete values (e.g. HeatMap) the axes will be labelled:

In [ ]:
hm = hv.HeatMap(adata, [A.obs['index'], A.var['index']], [A[:, :]])

hm.opts(responsive=True, height=800, xrotation=90)